# Bob's Discount Furniture

Warning: because some data products are inconsistent (changing JSON formats, etc...), not all code cleanly refactorable.

In [ ]:
from __future__ import division

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import xspec_utils as xs
from nice_tables import LatexTable

In [ ]:
fd = xs.load_dict("results_spec/20160701_fiveann.json")

## Soft proton parameters as a function of radius

In [ ]:
latex_hdr = [['Annulus', '2001 MOS', '2001 PN', '2009 MOS']]
latex_cols = ['{:s}', 0, 0, 0]
ltab = LatexTable(latex_hdr, latex_cols, "Soft proton power law indices", prec=2)

labels = ['0-100', '100-200', '200-300', '300-400', '400-500']
spectra = [np.array([1,3,5]),
           np.array([1,3,5]) + 5,
           np.array([1,3,5]) + 10,
           np.array([1,3,5]) + 15,
           np.array([1,3,5]) + 20]
spectra = [map(str, x) for x in spectra]

for lab, indices in zip(labels, spectra):
    ltr = [lab]
    for idx in indices:
        ltr.append(fd[idx]['sp']['powerlaw']['PhoIndex']['value'])
    ltab.add_row(*ltr)

print ltab

Hard to see for 0087940201 MOS1 case, but 200-300 powerlaw basically overlaps 400-500.
Behavior is broadly consistent with what we expect, actually, which is very reassuring.  Fits also get jumpier going outwards in radius.  But, outer annuli encompass larger annuli -- so the counts should still be pretty comparable.  I would actually expect outer annuli fits to be a bit better constrained

In [ ]:
x = np.logspace(-1, 1)

for offset, exposure in enumerate(['0087940201 MOS1', '0087940201 MOS2', '0087940201 PN',
                                   '0551000201 MOS1','0551000201 MOS2']):
    print exposure
    for idx, lab in zip(np.array([1, 6, 11, 16, 21]) + offset,
                        ['0-100', '100-200', '200-300', '300-400', '400-500']):
        idx = str(idx)
        phoindex = fd[idx]['sp']['powerlaw']['PhoIndex']['value']
        norm = fd[idx]['sp']['powerlaw']['norm']['value']
        print "{:s}: index {:.2f}, norm {:.2e}".format(lab, phoindex, norm)
        plt.loglog(x, norm * x**(-1 * phoindex), label=lab)

    plt.legend(loc='best')
    plt.xlabel("Energy (keV)")
    plt.ylabel(r'Effective photons s$^{-1}$ cm$^{-2}$ keV$^{-1}$')
    plt.show()

## vnei fit parameters as a function of radius

In [ ]:
fd = xs.load_dict("results_spec/20160701_fiveann.json")
vals = []
p_errs = []
n_errs = []
for idx, model in zip(np.array([1, 6, 11, 16, 21]),
                  ['snr_ann_000_100', 'snr_ann_100_200', 'snr_ann_200_300', 'snr_ann_300_400', 'snr_ann_400_500']):
    c = fd[str(idx)][model]['vnei']['kT']
    vals.append(c['value'])
    p_errs.append(c['error'][1] - c['value'])
    n_errs.append(abs(c['error'][0] - c['value']))
plt.errorbar([50, 150, 250, 350, 450], vals, yerr=[n_errs, p_errs], xerr=50, marker='o', ls='', label='5')

fd = xs.load_dict("results_spec/20160708_fourann_center-mg-o-free.json")
vals = []
p_errs = []
n_errs = []
for idx, model in zip(np.array([1, 6, 11, 16]),
                  ['snr_ann_000_100', 'snr_ann_100_200', 'snr_ann_200_300', 'snr_ann_300_400']):
    c = fd[str(idx)][model]['vnei']['kT']
    vals.append(c['value'])
    p_errs.append(c['error'][1] - c['value'])
    n_errs.append(abs(c['error'][0] - c['value']))
plt.errorbar([48, 148, 248, 348], vals, yerr=[n_errs, p_errs], xerr=50, marker='o', ls='', label='4Mg,O')

fd = xs.load_dict("results_spec/20160708_fourann_center-mg-fe-free.json")
vals = []
p_errs = []
n_errs = []
for idx, model in zip(np.array([1, 6, 11, 16]),
                  ['snr_ann_000_100', 'snr_ann_100_200', 'snr_ann_200_300', 'snr_ann_300_400']):
    c = fd[str(idx)][model]['vnei']['kT']
    vals.append(c['value'])
    p_errs.append(c['error'][1] - c['value'])
    n_errs.append(abs(c['error'][0] - c['value']))
plt.errorbar([52, 152, 252, 352], vals, yerr=[n_errs, p_errs], xerr=50, marker='o', ls='', label='4Mg,Fe')

fd = xs.load_dict("results_spec/20160708_fourann_center-mg-o-ne-free.json")
vals = []
p_errs = []
n_errs = []
for idx, model in zip(np.array([1, 6, 11, 16]),
                  ['snr_ann_000_100', 'snr_ann_100_200', 'snr_ann_200_300', 'snr_ann_300_400']):
    c = fd[str(idx)][model]['vnei']['kT']
    vals.append(c['value'])
    p_errs.append(c['error'][1] - c['value'])
    n_errs.append(abs(c['error'][0] - c['value']))
plt.errorbar([51, 151, 251, 351], vals, yerr=[n_errs, p_errs], xerr=50, marker='o', ls='', label='4Mg,O,Ne')

plt.legend(loc='best')
plt.xlabel("Radius (arcseconds)")
plt.ylabel(r'Plasma electron temperature (keV)')
plt.show()

## Source fit parameters

Todo: just regenerate all the joint source-background fits with new products.
Saves fair bit of trouble, should be straightforward.

## Four annuli with varied center abundances

In [ ]:
fdictfs = ["results_spec/20160708_fourann_center-mg-free_snr_ann_000_100.json",
          "results_spec/20160708_fourann_center-mg-o-free_snr_ann_000_100.json",
          "results_spec/20160708_fourann_center-mg-ne-free_snr_ann_000_100.json",
          "results_spec/20160708_fourann_center-mg-o-ne-free_snr_ann_000_100.json",
          "results_spec/20160708_fourann_center-mg-fe-free_snr_ann_000_100.json",
          "results_spec/20160712_fourann_center-mg-o-fe-free.json"
          ]
# fdictf = fit dict file
labels = ["4Mg", "4Mg,O", "4Mg,Ne", "4Mg,Ne,O", "4Mg,Fe", "4Mg,O,Fe"]

In [ ]:
def val_pnerrs(fit_dict, model, comp, old=False):
    """Convenience method to extract
    value, +ve error, -ve error
    from a fit_dict.  Errors are signed (i.e., -ve error value is < 0)
    
    AS CONSTRUCTED ONLY OPERATES ON FIRST SPECTRUM (hack) for non-"old" dicts
    
    Arguments
        fit_dict: xspec_utils fit dict (i.e., JSON dict)
        model: model name (string)
        comp: component name (string)
    Returns:
        value, +ve error, -ve error three-tuple
    """
    if not old:
        c = fit_dict["1"]["snr_ann_000_100"][model][comp]
    else:
        c = fit_dict['comps'][model][comp]
    val = c['value']
    pos_err = c['error'][1] - c['value']
    neg_err = c['error'][0] - c['value']
    return val, pos_err, neg_err

In [ ]:
latex_hdr = [['Annulus', ''],
             [r'$n_\mathrm{H}$', r'($10^{22} \unit{cm^{-2}}$)'],
             [r'$kT$', r'(keV)'],
             [r'$\tau$', r'($10^{10} \unit{s\;cm^{-3}}$)'],
             ['O', '(-)'],
             ['Ne', '(-)'],
             ['Mg', '(-)'],
             ['Si', '(-)'],
             ['S', '(-)'],
             ['Fe', '(-)'],
             [r'$\chi^2_{\mt{red}}$', r'$\chi^2 / (\mt{dof})$']]
latex_hdr = np.array(latex_hdr).T

latex_cols = ['{:s}', 2, 2, 2, 2, 2] + 4 * [2] + ['{:s}']  # O, Ne, Mg, Fe; chisqred
ltab = LatexTable(latex_hdr, latex_cols, "G309.2-0.6 annuli fit with errors", prec=2)

for fdictf, lab in zip(fdictfs, labels):
    
    fd = xs.load_dict(fdictf)
    old = "20160708" in fdictf
    
    ltr = [lab]
    ltr.extend(val_pnerrs(fd, 'tbnew_gas', 'nH', old=old))
    ltr.extend(val_pnerrs(fd, 'vnei', 'kT', old=old))
    ltr.extend(np.array(val_pnerrs(fd, 'vnei', 'Tau', old=old)) / 1e10)
    ltr.extend(val_pnerrs(fd, 'vnei', 'O', old=old))
    ltr.extend(val_pnerrs(fd, 'vnei', 'Ne', old=old))
    ltr.extend(val_pnerrs(fd, 'vnei', 'Mg', old=old))
    ltr.extend(val_pnerrs(fd, 'vnei', 'Si', old=old))
    ltr.extend(val_pnerrs(fd, 'vnei', 'S', old=old))
    ltr.extend(val_pnerrs(fd, 'vnei', 'Fe', old=old))
    
    if old:
        ltr.append("{:.3f} = {:.2f} / {:d}".format(fd['fitstat'][1] / fd['dof'], fd['fitstat'][1], fd['dof']))
    else:
        ltr.append("{:.3f} = {:.2f} / {:d}".format(fd['fitStat'] / fd['dof'], fd['fitStat'], fd['dof']))

    ltab.add_row(*ltr)

print ltab.__str__().replace('${1.00}^{-1.00}_{-1.00}$', '    ')

## Tabulate annulus fit parameters

In [ ]:
# Nice LaTeX table

fd = xs.load_dict("results_spec/20160701_fiveann.json")
rings = []
for idx, model in zip([1, 6, 11, 16, 21], ['snr_ann_000_100', 'snr_ann_100_200',
                                           'snr_ann_200_300', 'snr_ann_300_400', 'snr_ann_400_500']):
    rings.append(fd[str(idx)][model])

def ring_val_errs(ring, cname, pname):
    p = ring[cname][pname]
    val = p['value']
    pos_err = p['error'][1] - p['value']
    neg_err = p['error'][0] - p['value']
    return val, pos_err, neg_err

latex_hdr = [['Annulus', ''],
             [r'$n_\mathrm{H}$', r'($10^{22} \unit{cm^{-2}}$)'],
             [r'$kT$', r'(keV)'],
             [r'$\tau$', r'($10^{10} \unit{s\;cm^{-3}}$)'],
             ['Si', '(-)'],
             ['S', '(-)']]
latex_hdr = np.array(latex_hdr).T

latex_cols = ['{:s}', 2, 2, 2, 2, 2]
ltab = LatexTable(latex_hdr, latex_cols, "G309.2-0.6 annuli fit with errors", prec=2)

for ring in rings:
    ltr = [ring['name']]
    ltr.extend(ring_val_errs(ring, 'tbnew_gas', 'nH'))
    ltr.extend(ring_val_errs(ring, 'vnei', 'kT'))
    ltr.extend(np.array(ring_val_errs(ring, 'vnei', 'Tau')) / 1e10)
    ltr.extend(ring_val_errs(ring, 'vnei', 'Si'))
    ltr.extend(ring_val_errs(ring, 'vnei', 'S'))

    ltab.add_row(*ltr)

print ltab